##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

##### 第三模块(基于hanlp的绝对词频统计:必须运行)

In [3]:
# -*- coding:utf-8 -*-
# 绝对词频统计
pattern_word = re.compile(r".*?\\")
pattern_nature = re.compile(r"\\.*")

list_useful_words,list_natures = [],[]
def segments(article,stopwords):
    """
    param article:每篇文章
    param stopwords:停用词列表
    return list_articles:每篇文章的分词
    """
    for item in HanLP.segment(article):
        try:
            item_word = ('{}\{}'.format(item.word, item.nature))
            word = re.search(pattern_word,item_word).group()
            nature = re.search(pattern_nature,item_word).group()
            word_clean = word.strip("\\")
            nature_clean = nature.strip("\\")
            if word_clean not in list_stopwords:
                list_useful_words.append(word_clean)
                list_natures.append(nature_clean)
        except Exception as e:
            pass
    return list_useful_words,list_natures

def DF(list_useful_words,list_natures):
    """
    param list_useful_words:清洗后得到的词表
    param list_natures:清洗后词性的词性表
    return table_test_3:
    """
    table_test = pd.DataFrame([list_useful_words,list_natures]).T
    table_test.columns = ["word","nature"]
    
    ### 只提取名词和动词！！！！！！！！
    table_test_2 = table_test[table_test["nature"].isin(["n","v"])]
    table_test_3 = table_test_2.groupby("word").count().sort_values(["nature"],ascending = ["False"])
    return table_test_3

def main_word_freqs(corpus,list_stopwords):
    """
    param corpus:每个的文本
    param list_stopwords:
    return list_articles:每篇关键词的频率
    """
    list_articles = []
    for article in corpus[0:10]:
        try:
            list_useful_words,list_natures = segments(article,list_stopwords)
            table = DF(list_useful_words,list_natures)
            list_articles.append(table)
        except Exception as e:
            pass
    return list_articles

#### 第四模块(控制部分!)

In [4]:
### 频率部分控制！
### 懒得写__main__()了
### table_filter_regex.xlsx

def run_freqs(corpus,list_stopwords):
    corpus = list(filter(lambda x:x!=None,corpus))
    list_articles = main_word_freqs(corpus,list_stopwords)
    table_result = pd.concat(list_articles).groupby("word").sum().sort_values("nature",ascending = False)
    return table_result,list_articles

if __name__ == '__main__': 
    list_stopwords = []
    with open("stopwords.txt","r",encoding = "utf-8") as f:
        for word in f.readlines():
            list_stopwords.append(word.strip("\n"))
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_ready_to_eat = import_table(root)
    print("----------------------------------------------")
    print("******************表格已经导入******************")
    print("----------------------------------------------")
    print("表格中所有的列:",table_ready_to_eat.columns)
    selected_column = input("请问需要分析哪一列的关键词词频？(输入关键词部分):")
    corpus = table_ready_to_eat[selected_column]
    table_word_freqs,list_articles = run_freqs(corpus,list_stopwords)
    save_model(table_word_freqs)
# table_filter_regex

----------------------------------------------
******************表格已经导入******************
----------------------------------------------
表格中所有的列: Index(['标题', '案号', '案件类型', '庭审程序', '案由', '文书类型', '法院', '判决日期', '原告', '被告',
       '第三人', '法官', '审判长', '审判员', '书记员', '头部', '头部2', '当事人', '当事人2', '庭审程序说明',
       '庭审程序说明2', '庭审过程', '庭审过程2', '庭审过程3', '庭审过程4', '庭审过程5', '庭审过程6', '法院意见',
       '法院意见2', '判决结果', '判决结果2', '庭后告知', '庭后告知2', '结尾', '结尾2', '附录', '附录2'],
      dtype='object')
请问需要分析哪一列的关键词词频？(输入关键词部分):判决结果
-------------------------------------------
以下为表格的预览:
      nature
word        
原告       205
被告       173
负担        99
履行        76
诉讼法       72
是否需要保存该词频表？(输入Y/N):N
未能存储
